# Laboratorio 2: Clasificación




## Instrucciones

- Haz una copia de este notebook en `File -> Save a copy in Drive`.
- Al final de la sesión, descarga el notebook `File -> Download -> Download .ipynb` y súbelo en la sección Tareas de U-Cursos que se abrirá para esta sesión.


## Preguntas teóricas

### Pregunta 1

Los profesores del diplomado le han encargado a usted construir un clasificador usando los datos históricos de todos los diplomados pasados para predecir si los estudiantes aprobarán o no los cursos.

Usted tiene un dataset en formato tabular con información de la asistencia de los estudiantes, su género, edad, nota promedio de otros cursos, su cantidad de publicaciones en el foro, el número de clicks en los enlaces y la cantidad de mensajes que han enviado por correo. Además, tiene una columna con un 1 si el estudiante aprobó el curso y con un 0 si lo reprobó.


1. Considere que usted tiene acceso a todos los datos y probará distintos clasificadores para solucionar el problema. ¿Qué problemas podría haber con la distribución de las clases? ¿De qué manera lo solucionaría? Considere que los alumnos que reprueban son muy pocos en comparación a los que aprueban.

**Respuesta:**
El problema puede ser considerado de clasificación, pues la columna de aprobación es discreta.
El problema que podría existir asociado a la clase, es por su desbalance, pues los que reprueban (valor 0) son muy pocos comparados con los que aprueban (valor 1).
Para tratar de solucionar este posible problema, se pueden utilizar las técnicas de oversampling y undersampling en los datos de entrenamiento.  Al final, se deben comparar las métricas de desempeño, con y sin las técnicas utilizadas, y quedarse con el que tenga el desempeño más elevado.



2. El comité de ética solo ha permitido usar los datos que tengan consentimiento de los estudiantes. Sin embargo, quienes han dado su consentimiento son solo los estudiantes que tienen 30 años. ¿Qué problemas podría haber? ¿Se puede solucionar?

**Respuesta:**
El problema que podría existir es que los resultados podrían no representar lo que sucede con las otras edades, en otras palabras, la edad podría ser un atributo importante.
En ese caso no se podrían generalizar los resultados a las demás edades.

### Pregunta 2

Verdadero o Falso (si la afirmación es falsa justifique):

1. Mientras más features tenga un dataset, los clasificadores siempre obtienen mejores resultados en las predicciones.

**Respuesta:**
Falso, pues no todas las features pueden estar asociadas con la predicción que se quiera realizer. En algunos casos, un subconjunto más pequeño podría modelar mejor el problema. Además, algunas features pueden ser linealmente dependientes, por lo que esa relación directa sería equivalente a estar considerando datos redundates.


2. Precision, Recall y F1 son métricas que solamente funcionan para clasificadores que intentan predecir problemas con clases binarias y no sirven para problemas multiclases.

**Respuesta:**
Falso. Cuando la clase es multiclase, la matriz de confusión es "más grande". Para calcular las métricas asociadas hay dos técnicas, llamadas micro y macro averaging. Para hacer estos cálculos, se calculan las métricas usuales, para cada valor que pudiera tener la clase, "reduciendo" la matriz a una de 2x2.

3. La normalización de los datos es una técnica útil para evitar que atributos con valores muy grandes tengan demasiada importancia para los clasificadores.

**Respuesta:**
Verdadero.


## Preguntas prácticas

A continuación vamos a cargar datos que describen si el ingreso de una persona es superior a 50.000 dólares o no (columna `income_>50k`). El resto de los atributos presenta características de cada uno de los individuos.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving income.csv to income.csv


In [2]:
import io
import pandas as pd
import numpy as np
df = pd.read_csv(io.BytesIO(uploaded['income.csv']))

In [3]:
df.head(2)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43957 entries, 0 to 43956
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              43957 non-null  int64 
 1   workclass        41459 non-null  object
 2   fnlwgt           43957 non-null  int64 
 3   education        43957 non-null  object
 4   educational-num  43957 non-null  int64 
 5   marital-status   43957 non-null  object
 6   occupation       41451 non-null  object
 7   relationship     43957 non-null  object
 8   race             43957 non-null  object
 9   gender           43957 non-null  object
 10  capital-gain     43957 non-null  int64 
 11  capital-loss     43957 non-null  int64 
 12  hours-per-week   43957 non-null  int64 
 13  native-country   43194 non-null  object
 14  income_>50K      43957 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 5.0+ MB


### Pregunta 3

Para las columnas de tipo string, transforme el valor de sus variables a uno de tipo entero (int). Considere que a partir de ahora, este será el dataset que utilizará. **Hint:** Puede aplicar `LabelEncoder()` o `get_dummies()`.

In [5]:
#Se identifican qué columnas son string, para eso se exluyen las enteras (no existen float).
str_columns = list(df.select_dtypes(exclude=['int64']).columns)
df.loc[:,str_columns].head(3)

,workclass,education,marital-status,occupation,relationship,race,gender,native-country
0,Private,Doctorate,Divorced,Exec-managerial,Not-in-family,White,Male,United-States
1,Private,12th,Never-married,Other-service,Own-child,White,Male,United-States
2,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States


In [6]:
#En este caso, también se podría haber realizado la operación con include en vez de exclude.
str_columns = list(df.select_dtypes(include=['object']).columns)
df.loc[:,str_columns].head(3)

,workclass,education,marital-status,occupation,relationship,race,gender,native-country
0,Private,Doctorate,Divorced,Exec-managerial,Not-in-family,White,Male,United-States
1,Private,12th,Never-married,Other-service,Own-child,White,Male,United-States
2,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States


In [7]:
#Se importa LabelEncoder
from sklearn.preprocessing import LabelEncoder
#Se aplica la transformación
df.loc[:,str_columns] = df.loc[:,str_columns].apply(LabelEncoder().fit_transform)

<ipython-input-7-8eba35fd26f3>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,str_columns] = df.loc[:,str_columns].apply(LabelEncoder().fit_transform)


In [8]:
#Se verifica que se realizó la transformación
df.head(2)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,3,366425,10,16,0,3,1,4,1,99999,0,60,38,1
1,17,3,244602,2,8,4,7,3,4,1,0,0,15,38,0


### Pregunta 4

Separe el dataset en datos de entrenamiento y prueba considerando un 20% de los datos totales para testing. Mantenga la distribución de la clase objetivo entre los distintos conjuntos. Almacene estos valores pues serán utilizados en las siguientes preguntas.

In [9]:
#Se importa train_test_split para separar los datos
from sklearn.model_selection import train_test_split

#Se extraen los atributos (X) y la clase (y) desde el dataset (df)
X = np.array(df.iloc[:,:-1])      #todas las columnas excepto la última
y = df.iloc[: , -1]               #correspondiente a la columna income_>50k

#Se dividen los datos en training y testing, en este caso un 80% y 20%, respectivamente, (test_size=0.2)
#Para mantener la distribución de clase objetivo entre los distintos conjuntos se utiliza stratify=clase
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=36, stratify=y)

### Pregunta 5


Utilizando solo los datos de entrenamiento, aplique `GridSearchCV` sobre al menos 2 algoritmos de clasificación con diferentes hiperparámetros. Obtenga aquel que posea el mayor F1-score al predecir la columna `income_>50K`.

Para cada algoritmo de clasificación utilizado, señale cuáles fueron los mejores hiperparámetros obtenidos junto a su F1-score.

Mencione además qué valor de `k` usó para k-fold cross-validation.

Puede ocupar clasificadores tales como:

* [Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
* [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [Support Vector Machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
* [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)
* [Multi-layer Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)
*  O algún otro modelo que usted desee o se haya visto en clases.


In [10]:
#Se importa GridSearchCV, que permitirá realizar un ajuste de hiperparámetros para determinar
#los valores óptimos para un modelo determinado.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [11]:
#El valor usado para el k-fold cross-validation es k=4
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=4)
#Nos interesa calcular el mayor F1-score
scoring = 'f1'

In [12]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
parameters = {'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]}
gs_NB  = GridSearchCV(GaussianNB(), parameters, cv=cv,scoring=scoring)
gs_NB.fit(X_train, y_train)


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=GaussianNB(),
             param_grid={'var_smoothing': [0.01, 0.001, 0.0001, 1e-05, 1e-06,
                                           1e-07, 1e-08, 1e-09, 1e-10, 1e-11,
                                           1e-12, 1e-13, 1e-14, 1e-15]},
             scoring='f1')

In [13]:
gs_NB.best_params_

{'var_smoothing': 1e-14}

In [14]:
gs_NB.best_score_

0.4538395457115434

In [15]:
#Adaboost
#Un hiperparámetro importante es el n_estimator (los valores van entre [1, inf), el valor por defecto es 50)
from sklearn.ensemble import AdaBoostClassifier
n_range = list(range(40, 60))
param_grid = {'n_estimators': n_range}
gs_ada = GridSearchCV(AdaBoostClassifier(), param_grid, cv = cv, scoring=scoring)
gs_ada.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=AdaBoostClassifier(),
             param_grid={'n_estimators': [40, 41, 42, 43, 44, 45, 46, 47, 48,
                                          49, 50, 51, 52, 53, 54, 55, 56, 57,
                                          58, 59]},
             scoring='f1')

In [16]:
gs_ada.best_params_

{'n_estimators': 58}

In [17]:
gs_ada.best_score_

0.6814311554976158

In [18]:
#El hiperparámetro más importante de KNN es el número de vecinos.
from sklearn.neighbors import KNeighborsClassifier
param_grid={'n_neighbors':[3,4,5,6], 'weights': ['uniform', 'distance']}
gs_KN = GridSearchCV(KNeighborsClassifier(), param_grid, cv = cv, scoring=scoring)
gs_KN.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 4, 5, 6],
                         'weights': ['uniform', 'distance']},
             scoring='f1')

In [19]:
gs_KN.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [20]:
gs_KN.best_score_

0.44453629854459986

In [21]:
from sklearn.ensemble import RandomForestClassifier
param_grid={'n_estimators':[10, 50, 100, 150], 'criterion':['gini', 'entropy', 'log_loss']}
gs_randomF = GridSearchCV(RandomForestClassifier(), param_grid, cv = cv, scoring=scoring)
gs_randomF.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'n_estimators': [10, 50, 100, 150]},
             scoring='f1')

In [22]:
gs_randomF.best_params_

{'criterion': 'log_loss', 'n_estimators': 150}

In [23]:
gs_randomF.best_score_

0.677269798689559

## Pregunta 6

Usando el mejor modelo de clasificación obtenido previamente sobre los datos de entrenamiento, estime las métricas de clasificación en los datos de prueba (testing). Comente acerca de los resultados de la clasificación sobre la clase positiva y negativa.

In [28]:
#El mejor modelo fue Adaboost con parámetro n_estimators=58
clf=AdaBoostClassifier(n_estimators=58)
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      6688
           1       0.76      0.60      0.67      2104

    accuracy                           0.86      8792
   macro avg       0.82      0.77      0.79      8792
weighted avg       0.85      0.86      0.85      8792



- Se observa que todas las métricas (precision, recall y f1-score) son mayores para la clase 0 que para la clase 1.
- Esto se puede deber al desbalance que existe entre las clases, como se observa en support.
- En los datos de testing, existen 6688 registros para la clase 0, mientras que sólo 2104 para la clase 1, es decir, un tercio aproximadamente.
- Para mejorar las métricas de la clase 1 podría ser necesario aplicar las técnicas de oversampling y undersampling, en los datos de entrenamiento.